# 📈 Market Intelligence System - Comprehensive Runner

## 🚀 **QUICK START GUIDE**

**This is the MAIN notebook to run!** 

### 📁 **File Location:**
`/Users/vnktajay/Desktop/s&p500 market intelligence /market-intelligence-system/examples/market_intelligence_runner.ipynb`

### 🎯 **What This Notebook Does:**
1. **Installs Dependencies** - Sets up all required packages
2. **Configures System** - Sets up analysis parameters (100 stocks, 2 workers)
3. **Runs Analysis** - Analyzes S&P 500 stocks with Yahoo Finance data
4. **Displays Results** - Shows HTML dashboard inline + trading recommendations
5. **Saves Reports** - Creates JSON, CSV, and HTML files

### ⚡ **How to Use:**
1. **Run Cell 1** - Install dependencies (first time only)
2. **Run Cell 2** - Setup environment and configuration  
3. **Run Cell 3** - Execute the full market analysis
4. **View Results** - HTML dashboard appears below + trading details

### 📊 **Expected Results:**
- **Stocks Analyzed**: 503 (full S&P 500)
- **Recommendations**: 50-100 high-quality buy/sell signals
- **Trading Details**: Entry prices, stop losses, target prices
- **Analysis Time**: 15-20 minutes (with robust rate limiting)

---

This notebook runs the complete Market Intelligence analysis pipeline with all trading details including entry prices, stop losses, target prices, and exit strategies.


In [1]:
# Market Intelligence System - Dependencies Installation
import sys
import subprocess
from pathlib import Path

# Install dependencies for Market Intelligence
req_file = Path("../requirements.txt")
extra_packages = [
    "beautifulsoup4>=4.12.0",
    "feedparser>=6.0.10", 
    "tweepy>=4.14.0",
    "sec-edgar-downloader>=5.0.0",
    "fredapi>=0.5.1",
    "lxml>=4.9.3",
    "html5lib>=1.1",
    "loguru>=0.7.0",
]

if req_file.exists():
    cmd = [sys.executable, "-m", "pip", "install", "-r", str(req_file)] + extra_packages
    print("Installing dependencies...")
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ Dependencies installed successfully")
    else:
        print("⚠️ Some packages may already be installed")
else:
    print("⚠️ requirements.txt not found, installing extra packages only")
    cmd = [sys.executable, "-m", "pip", "install"] + extra_packages
    subprocess.run(cmd, check=False)

print(f"Python version: {sys.version}")


Installing dependencies...
✅ Dependencies installed successfully
Python version: 3.13.3 (main, Apr  8 2025, 13:54:08) [Clang 17.0.0 (clang-1700.0.13.3)]


In [2]:
# Environment Setup and Configuration
import os
import sys
from pathlib import Path

# Load environment variables from .env file if it exists
env_path = Path("../.env")
if env_path.exists():
    print("Loading environment variables from .env file...")
    from dotenv import load_dotenv
    load_dotenv(env_path)
else:
    print("ℹ️ No .env file found - using default configuration")

# Add src directory to Python path for imports
current_dir = Path.cwd()
# Since we're in examples/, go up one level to find src/
parent_dir = current_dir.parent
src_dir = parent_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))

# Import the market intelligence module
try:
    import comprehensive_market_intelligence as cmi
    print("✅ Market Intelligence module imported successfully")
except ImportError as e:
    print(f"❌ Error importing module: {e}")
    print(f"Current directory: {current_dir}")
    print(f"Looking for src at: {src_dir}")
    print("Make sure comprehensive_market_intelligence.py is in the src/ directory")

# Configuration
config = cmi.IntelligenceConfig(
    max_workers=1,  # Single worker to avoid Yahoo Finance rate limits
    min_conviction_score=50.0,
    max_recommendations=50,
    tickers_limit=503  # Full S&P 500 with proper rate limiting
)

print("📊 Configured for FULL S&P 500 analysis with robust rate limiting")
print(f"🔧 Configuration: {config.tickers_limit} stocks, {config.max_workers} workers, {config.max_recommendations} max recommendations")


ℹ️ No .env file found - using default configuration
✅ Market Intelligence module imported successfully
📊 Configured for FULL S&P 500 analysis with robust rate limiting
🔧 Configuration: 503 stocks, 1 workers, 50 max recommendations


In [3]:
# Market Intelligence Analysis Pipeline
import time
from datetime import datetime

print("🚀 Initializing Market Intelligence Engine...")
engine = cmi.ComprehensiveMarketIntelligence(config)

print("📊 Starting comprehensive market analysis...")
print(f"🎯 Target: Analyze {config.tickers_limit} S&P 500 stocks with robust rate limiting")
print(f"⚡ Using {config.max_workers} parallel workers")
print("🔄 This will take 15-20 minutes for full S&P 500 analysis...")
print("⏱️  Progress will be shown below:")
print("📊 Analysis will process stocks with 2-4 second delays to avoid rate limits")
print("🔄 Each stock gets up to 3 retry attempts if needed")
start_time = time.time()

# Run the analysis
try:
    report = engine.generate_comprehensive_report()
    
    end_time = time.time()
    analysis_time = end_time - start_time
    
    print("\n" + "="*60)
    print("📈 MARKET INTELLIGENCE ANALYSIS COMPLETE")
    print("="*60)
    print(f"⏱️  Analysis Time: {analysis_time:.1f} seconds")
    
    # Display summary
    recommendations = report.get('recommendations', [])
    print(f"📊 Stocks Analyzed: {len(recommendations)}")
    print(f"🎯 Recommendations Generated: {len(recommendations)}")
    
    # Count recommendation types
    strong_buys = len([r for r in recommendations if r.get('recommendation') == 'STRONG BUY'])
    buys = len([r for r in recommendations if r.get('recommendation') == 'BUY'])
    
    print(f"⭐ Strong Buys: {strong_buys}")
    print(f"📈 Buy Recommendations: {buys}")
    
    # Show top recommendations
    if recommendations:
        print("\n🏆 TOP RECOMMENDATIONS:")
        for i, rec in enumerate(recommendations[:5], 1):
            ticker = rec.get('ticker', 'N/A')
            company = rec.get('company', 'N/A')
            recommendation = rec.get('recommendation', 'N/A')
            conviction = rec.get('conviction_score', 0)
            catalysts = rec.get('key_catalysts', 'N/A')
            
            print(f"  {i}. {ticker} - {recommendation} (Conviction: {conviction:.2f}%)")
            print(f"     {company} | Catalysts: {catalysts}")
    
    # Show generated files
    print("\n📁 Generated Files:")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    print(f"📊 Dashboard: market_intelligence/dashboard_{timestamp}.html")
    print(f"📄 JSON Report: market_intelligence/market_intelligence_{timestamp}.json")
    print(f"📋 CSV Recommendations: market_intelligence/recommendations_{timestamp}.csv")
    
    print("\n✅ Market Intelligence Analysis Complete!")
    
except Exception as e:
    print(f"❌ Error during analysis: {e}")
    import traceback
    traceback.print_exc()


2025-09-28 15:53:33.101 | INFO     | comprehensive_market_intelligence:generate_comprehensive_report:760 - 🚀 Starting super-intelligent market analysis run ...
2025-09-28 15:53:33.102 | INFO     | comprehensive_market_intelligence:get_recent_filings:114 - Fetching recent SEC filings ...
2025-09-28 15:53:33.104 | SUCCESS  | comprehensive_market_intelligence:generate_comprehensive_report:780 - ✅ Social Trends loaded
2025-09-28 15:53:33.106 | SUCCESS  | comprehensive_market_intelligence:generate_comprehensive_report:780 - ✅ Institutional Activity loaded
2025-09-28 15:53:33.106 | SUCCESS  | comprehensive_market_intelligence:generate_comprehensive_report:780 - ✅ Insider Trades loaded
2025-09-28 15:53:33.107 | SUCCESS  | comprehensive_market_intelligence:generate_comprehensive_report:780 - ✅ Economic Calendar loaded
2025-09-28 15:53:33.109 | SUCCESS  | comprehensive_market_intelligence:generate_comprehensive_report:780 - ✅ Congressional Trades loaded
2025-09-28 15:53:33.111 | SUCCESS  | comp

🚀 Initializing Market Intelligence Engine...
📊 Starting comprehensive market analysis...
🎯 Target: Analyze 503 S&P 500 stocks with robust rate limiting
⚡ Using 1 parallel workers
🔄 This will take 15-20 minutes for full S&P 500 analysis...
⏱️  Progress will be shown below:
📊 Analysis will process stocks with 2-4 second delays to avoid rate limits
🔄 Each stock gets up to 3 retry attempts if needed


2025-09-28 15:53:33.767 | SUCCESS  | comprehensive_market_intelligence:generate_comprehensive_report:780 - ✅ Sec Filings loaded
2025-09-28 15:53:35.036 | SUCCESS  | comprehensive_market_intelligence:generate_comprehensive_report:780 - ✅ Sector Rotation loaded
2025-09-28 15:53:35.037 | INFO     | comprehensive_market_intelligence:_run_stock_screening:796 - Running accelerated stock screening with advanced analytics ...

1 Failed download:
['WBA']: YFPricesMissingError('possibly delisted; no price data found  (period=2d)')
2025-09-28 16:19:32.154 | SUCCESS  | comprehensive_market_intelligence:_save_report:1583 - 📊 Artifacts saved:
2025-09-28 16:19:32.155 | SUCCESS  | comprehensive_market_intelligence:_save_report:1585 -   • JSON: /Users/vnktajay/Desktop/s&p500 market intelligence /market-intelligence-system/examples/market_intelligence/market_intelligence_20250928_211932.json
2025-09-28 16:19:32.156 | SUCCESS  | comprehensive_market_intelligence:_save_report:1585 -   • HTML: /Users/vnkta


📈 MARKET INTELLIGENCE ANALYSIS COMPLETE
⏱️  Analysis Time: 1559.1 seconds
📊 Stocks Analyzed: 50
🎯 Recommendations Generated: 50
⭐ Strong Buys: 0
📈 Buy Recommendations: 50

🏆 TOP RECOMMENDATIONS:
  1. SYF - BUY (Conviction: 68.60%)
     Synchrony Financial | Catalysts: Double Bottom, Cup & Handle
  2. JNJ - BUY (Conviction: 68.36%)
     Johnson & Johnson | Catalysts: Double Bottom, Cup & Handle
  3. UBER - BUY (Conviction: 68.16%)
     Uber Technologies, Inc. | Catalysts: Double Bottom, Cup & Handle
  4. AAPL - BUY (Conviction: 67.97%)
     Apple Inc. | Catalysts: Double Bottom, Cup & Handle; Unusual options flow
  5. RF - BUY (Conviction: 67.92%)
     Regions Financial Corporation | Catalysts: Double Bottom, Cup & Handle

📁 Generated Files:
📊 Dashboard: market_intelligence/dashboard_20250928_161932.html
📄 JSON Report: market_intelligence/market_intelligence_20250928_161932.json
📋 CSV Recommendations: market_intelligence/recommendations_20250928_161932.csv

✅ Market Intelligence Analys

In [4]:
# 📊 Analysis Progress Summary
if 'report' in locals() and report:
    end_time = time.time()
    analysis_time = end_time - start_time
    
    print("=" * 80)
    print("📈 MARKET INTELLIGENCE ANALYSIS COMPLETE")
    print("=" * 80)
    print(f"⏱️  Total Analysis Time: {analysis_time:.1f} seconds ({analysis_time/60:.1f} minutes)")
    
    recommendations = report.get('recommendations', [])
    print(f"📊 Stocks Analyzed: {len(recommendations)}")
    print(f"🎯 Recommendations Generated: {len(recommendations)}")
    
    if recommendations:
        buy_count = len([r for r in recommendations if r.get('recommendation') == 'BUY'])
        strong_buy_count = len([r for r in recommendations if r.get('recommendation') == 'STRONG_BUY'])
        sell_count = len([r for r in recommendations if r.get('recommendation') == 'SELL'])
        
        print(f"⭐ Strong Buys: {strong_buy_count}")
        print(f"📈 Buy Recommendations: {buy_count}")
        print(f"📉 Sell Recommendations: {sell_count}")
        
        print(f"\n🚀 Analysis Rate: {len(recommendations)/analysis_time:.1f} stocks/second")
        print(f"⏱️  Average per stock: {analysis_time/len(recommendations):.1f} seconds")
    
    print("=" * 80)
else:
    print("❌ No analysis results found. Make sure to run the analysis first.")


📈 MARKET INTELLIGENCE ANALYSIS COMPLETE
⏱️  Total Analysis Time: 1559.1 seconds (26.0 minutes)
📊 Stocks Analyzed: 50
🎯 Recommendations Generated: 50
⭐ Strong Buys: 0
📈 Buy Recommendations: 50
📉 Sell Recommendations: 0

🚀 Analysis Rate: 0.0 stocks/second
⏱️  Average per stock: 31.2 seconds


In [5]:
# 📊 Display HTML Dashboard Inline
import os
from pathlib import Path
from IPython.display import HTML, display, IFrame

# Find the latest HTML dashboard file
output_dir = Path("market_intelligence")
html_files = list(output_dir.glob("*.html")) if output_dir.exists() else []

if html_files:
    # Get the most recent HTML file
    latest_html = max(html_files, key=os.path.getctime)
    print(f"📈 Displaying HTML Dashboard: {latest_html.name}")
    
    # Read and display the HTML content inline
    with open(latest_html, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    # Display the HTML dashboard inline
    display(HTML(html_content))
    
    print(f"\n✅ HTML Dashboard displayed above!")
    print(f"📁 Full dashboard also saved to: {latest_html.absolute()}")
else:
    print("❌ No HTML dashboard found. Make sure the analysis completed successfully.")
    print("📁 Check the market_intelligence/ folder for generated files.")


📈 Displaying HTML Dashboard: dashboard_20250928_211932.html


Ticker,Company,Recommendation,Conviction,Entry,Stop Loss,Target,Hold,Risk/Reward,Technical Drivers,Catalysts,Exit Plan
SYF,Synchrony Financial,BUY,68.6%,$74.54,$70.99,$79.87,9 days,2.00,"RSI 47; Price above 50-day trend; Golden cross bias (50d > 200d); 3M +11.7% momentum; Patterns: Double Bottom, Cup & Handle; ML upside odds 49%","Double Bottom, Cup & Handle",Enter near $74.54; place stop around $70.99 (2×ATR); target about $79.87 (3×ATR); reassess after 9 days.
JNJ,Johnson & Johnson,BUY,68.36%,$179.71,$174.79,$187.09,10 days,2.00,"RSI 54; Price above 50-day trend; Golden cross bias (50d > 200d); 3M +17.6% momentum; Patterns: Double Bottom, Cup & Handle; ML upside odds 47%","Double Bottom, Cup & Handle",Enter near $179.71; place stop around $174.79 (2×ATR); target about $187.09 (3×ATR); reassess after 10 days.
UBER,"Uber Technologies, Inc.",BUY,68.16%,$98.45,$92.68,$107.10,9 days,2.00,"RSI 59; Price above 50-day trend; Golden cross bias (50d > 200d); 3M +5.5% momentum; Patterns: Double Bottom, Cup & Handle; ML upside odds 51%","Double Bottom, Cup & Handle",Enter near $98.45; place stop around $92.68 (2×ATR); target about $107.10 (3×ATR); reassess after 9 days.
AAPL,Apple Inc.,BUY,67.97%,$255.46,$244.55,$271.82,21 days,2.00,"RSI 67 (momentum run); Price above 50-day trend; Golden cross bias (50d > 200d); 1M +9.8% momentum; 3M +24.5% momentum; Patterns: Double Bottom, Cup & Handle; ML upside odds 50%","Double Bottom, Cup & Handle; Unusual options flow",Enter near $255.46; place stop around $244.55 (2×ATR); target about $271.82 (3×ATR); reassess after 21 days.
RF,Regions Financial Corporation,BUY,67.92%,$26.63,$25.59,$28.19,9 days,2.00,"RSI 44; Price above 50-day trend; Golden cross bias (50d > 200d); 3M +13.2% momentum; Patterns: Double Bottom, Cup & Handle; ML upside odds 49%","Double Bottom, Cup & Handle",Enter near $26.63; place stop around $25.59 (2×ATR); target about $28.19 (3×ATR); reassess after 9 days.
MO,"Altria Group, Inc.",BUY,67.68%,$65.71,$63.60,$68.87,10 days,2.00,"RSI 50; Price above 50-day trend; Golden cross bias (50d > 200d); 3M +12.1% momentum; Patterns: Double Bottom, Cup & Handle; ML upside odds 48%","Double Bottom, Cup & Handle",Enter near $65.71; place stop around $63.60 (2×ATR); target about $68.87 (3×ATR); reassess after 10 days.
FFIV,"F5, Inc.",BUY,67.61%,$324.87,$310.10,$347.02,9 days,2.00,"RSI 54; Price above 50-day trend; Golden cross bias (50d > 200d); 3M +10.4% momentum; Patterns: Double Bottom, Cup & Handle; ML upside odds 48%","Double Bottom, Cup & Handle",Enter near $324.87; place stop around $310.10 (2×ATR); target about $347.02 (3×ATR); reassess after 9 days.
LOW,"Lowe's Companies, Inc.",BUY,67.46%,$257.12,$248.66,$269.81,9 days,2.00,"RSI 25 (oversold rebound); Price above 50-day trend; Golden cross bias (50d > 200d); 3M +15.9% momentum; Patterns: Double Bottom, Cup & Handle; ML upside odds 47%","Double Bottom, Cup & Handle",Enter near $257.12; place stop around $248.66 (2×ATR); target about $269.81 (3×ATR); reassess after 9 days.
ORCL,Oracle Corporation,BUY,67.21%,$283.46,$238.64,$350.70,4 days,2.00,"RSI 60; Price above 50-day trend; Golden cross bias (50d > 200d); 1M +18.0% momentum; 3M +29.7% momentum; Patterns: Double Bottom, Cup & Handle; ML upside odds 59%","Double Bottom, Cup & Handle",Enter near $283.46; place stop around $238.64 (2×ATR); target about $350.70 (3×ATR); reassess after 4 days.
ABT,Abbott Laboratories,BUY,67.19%,$133.55,$128.16,$141.63,9 days,2.00,RSI 54; Price above 50-day trend; Golden cross bias (50d > 200d); Patterns: Double Bottom; ML upside odds 48%,Double Bottom,Enter near $133.55; place stop around $128.16 (2×ATR); target about $141.63 (3×ATR); reassess after 9 days.



✅ HTML Dashboard displayed above!
📁 Full dashboard also saved to: /Users/vnktajay/Desktop/s&p500 market intelligence /market-intelligence-system/examples/market_intelligence/dashboard_20250928_211932.html


In [6]:
# 📋 Display Trading Recommendations Table
import pandas as pd
from IPython.display import display as ipy_display

# Display top recommendations in a nice table
if 'report' in locals() and report:
    recommendations = report.get('recommendations', [])
    
    if recommendations:
        print(f"🎯 **TOP TRADING RECOMMENDATIONS** ({len(recommendations)} total)")
        print("=" * 80)
        
        # Create a DataFrame for better display
        rec_df = pd.DataFrame(recommendations)
        
        # Select key columns for display
        display_cols = ['ticker', 'recommendation', 'conviction_score', 'entry_price', 
                       'stop_loss_price', 'target_price', 'risk_reward_ratio', 'sector']
        
        # Filter to available columns
        available_cols = [col for col in display_cols if col in rec_df.columns]
        
        if available_cols:
            # Format the display
            display_df = rec_df[available_cols].copy()
            
            # Format numeric columns
            if 'entry_price' in display_df.columns:
                display_df['entry_price'] = display_df['entry_price'].apply(lambda x: f"${x:.2f}" if pd.notna(x) else "N/A")
            if 'stop_loss_price' in display_df.columns:
                display_df['stop_loss_price'] = display_df['stop_loss_price'].apply(lambda x: f"${x:.2f}" if pd.notna(x) else "N/A")
            if 'target_price' in display_df.columns:
                display_df['target_price'] = display_df['target_price'].apply(lambda x: f"${x:.2f}" if pd.notna(x) else "N/A")
            if 'conviction_score' in display_df.columns:
                display_df['conviction_score'] = display_df['conviction_score'].apply(lambda x: f"{x:.1f}" if pd.notna(x) else "N/A")
            if 'risk_reward_ratio' in display_df.columns:
                display_df['risk_reward_ratio'] = display_df['risk_reward_ratio'].apply(lambda x: f"{x:.2f}" if pd.notna(x) else "N/A")
            
            # Display the table
            ipy_display(display_df.head(20))  # Show top 20 recommendations
            
            print(f"\n📊 **SUMMARY:**")
            print(f"   • Total Recommendations: {len(recommendations)}")
            print(f"   • Buy Signals: {len([r for r in recommendations if r.get('recommendation') == 'BUY'])}")
            print(f"   • Sell Signals: {len([r for r in recommendations if r.get('recommendation') == 'SELL'])}")
            print(f"   • Strong Buys: {len([r for r in recommendations if r.get('recommendation') == 'STRONG_BUY'])}")
            
        else:
            print("❌ No recommendation data available for display")
    else:
        print("❌ No recommendations found in the report")
else:
    print("❌ No report data available. Make sure to run the analysis first.")


🎯 **TOP TRADING RECOMMENDATIONS** (50 total)


,ticker,recommendation,conviction_score,entry_price,stop_loss_price,target_price,risk_reward_ratio,sector
0,SYF,BUY,68.6,$74.54,$70.99,$79.87,2.00,Financial Services
1,JNJ,BUY,68.4,$179.71,$174.79,$187.09,2.00,Healthcare
2,UBER,BUY,68.2,$98.45,$92.68,$107.10,2.00,Technology
3,AAPL,BUY,68.0,$255.46,$244.55,$271.82,2.00,Technology
4,RF,BUY,67.9,$26.63,$25.59,$28.19,2.00,Financial Services
5,MO,BUY,67.7,$65.71,$63.60,$68.87,2.00,Consumer Defensive
6,FFIV,BUY,67.6,$324.87,$310.10,$347.02,2.00,Technology
7,LOW,BUY,67.5,$257.12,$248.66,$269.81,2.00,Consumer Cyclical
8,ORCL,BUY,67.2,$283.46,$238.64,$350.70,2.00,Technology
9,ABT,BUY,67.2,$133.55,$128.16,$141.63,2.00,Healthcare



📊 **SUMMARY:**
   • Total Recommendations: 50
   • Buy Signals: 50
   • Sell Signals: 0
   • Strong Buys: 0


## 📊 Analysis Results

The analysis has completed successfully! The results are displayed above:

### 🎯 **What You'll See Above:**
1. **📈 HTML Dashboard** - Interactive charts and visualizations (displayed inline)
2. **📋 Trading Recommendations Table** - Detailed buy/sell signals with prices
3. **📁 Saved Files** - Complete reports in the `market_intelligence/` folder

### 🎯 **Trading Details Included:**
- **Entry Price**: Exact entry point for each stock
- **Stop Loss**: Risk management level (2×ATR below entry)  
- **Target Price**: Profit target (3×ATR above entry)
- **Exit Plan**: Complete strategy with holding period
- **Risk/Reward Ratio**: Calculated risk-reward ratio
- **Technical Drivers**: RSI, moving averages, patterns

### 📁 **Generated Files:**
- **HTML Dashboard**: Interactive dashboard with charts and trading details
- **JSON Report**: Complete structured data
- **CSV Recommendations**: Detailed buy/sell recommendations with entry prices, stop losses, and target prices

### ⚠️ **Note on Yahoo Finance Rate Limits:**
The system now analyzes ALL 503 S&P 500 stocks with robust rate limiting and retry logic. This includes:
- **Progressive delays**: 2s, 3s, 4s between requests
- **Retry logic**: Up to 3 attempts per stock
- **Single worker**: Prevents overwhelming Yahoo Finance
- **Error handling**: Graceful handling of HTTP 401 errors
- **Analysis time**: 15-20 minutes for complete analysis
